In [ ]:
# to be installed 
#pip install -U transformers
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#pip install bitsandbytes
#pip install accelerate 
#conda install langchain -c conda-forge
#conda install -c huggingface -c conda-forge datasets
#pip install faiss-cpu
#pip install ipywidgets widgetsnbextension pandas-profiling
#pip install jupyter
#pip install PyPDF2
#pip install -qU langchain_community pypdf
#pip install -U sentence-transformers
#pip install openai







In [1]:
#Load libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import AgentExecutor
import faiss
from langchain_community.document_loaders import PyPDFLoader



# Step 1 Document ingestion

redact private infor 
text preprocessing 

In [ ]:

#lnitialize pdf loader
pdf_loader = PyPDFLoader("hr_manual.pdf")

#load documents
hr_manual = pdf_loader.load()

#preview document
print(f"loaded document with {len(hr_manual)} pages from the pdf")

#preview first 500 characters
print(hr_manual[20].page_content[:500])

#data type
print(type(hr_manual))


In [ ]:
type(hr_manual[0]) #this file type is immutable

In [ ]:
#for privacy concerns : remove metadata

def sanitize_metadata(document):
    from langchain.schema import Document

    ''' redacts metadata from document'''

    #create a redacted copy of meta data
    sanitize_metadata = document.metadata.copy()

    #remove sensitive data 
    sensitive_fields = ['producer','creator','author']


    for field in sensitive_fields:
        if field in sanitize_metadata:
            sanitize_metadata[field] = "[REDACTED]"

    # Create a new Document instance with the redacted metadata
    sanitized_doc = Document(page_content=document.page_content,
                             metadata=sanitize_metadata
                             )

    return sanitized_doc




In [ ]:
sanitized_hr_manual = [sanitize_metadata(doc) for doc in hr_manual]

#Check senstive data redacted from document's metadata
print(sanitized_hr_manual[25].metadata)

# Step 2 Create Embedings and Faiss Index

In [ ]:
emb_model_id = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {"normalize_embeddings": False}

#initialize embedding model
embedding_model= HuggingFaceEmbeddings(model_name=emb_model_id,
                                       model_kwargs= model_kwargs,
                                       encode_kwargs=encode_kwargs
                                       )


In [ ]:
#create Faiss Index

faiss_index = FAISS.from_documents(sanitized_hr_manual,embedding_model)

# Step 3: Test DistilBERT for Question Answering

In [ ]:
#from langchain.llms import HuggingFacePipeline
#from transformers import pipeline
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

In [3]:
#Load LLM model 
llm_model_id = 'distilbert-base-uncased-distilled-squad'
tokenizer = 'distilbert-base-uncased-distilled-squad'

llm_model = DistilBertForQuestionAnswering.from_pretrained(llm_model_id,
                                                           device_map="cuda",
                                                           return_dict=True,
                                                           torch_dtype=torch.bfloat16
                                                           )


#initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer)


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

c:\Users\On3B3\anaconda3\envs\prodenv3\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\On3B3\.cache\huggingface\hub\models--distilbert-base-uncased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

In [5]:

device = llm_model.device
inputs = tokenizer(question, text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = llm_model(**inputs)

In [6]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

# target is "nice puppet"
target_start_index = outputs.start_logits.argmax().unsqueeze(0)
target_end_index = outputs.end_logits.argmax().unsqueeze(0)

outputs = llm_model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss

In [7]:


predict_answer_tokens = inputs.input_ids[0, answer_start_index:answer_end_index + 1]

answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

print("Predicted Answer:", answer)

Predicted Answer: a nice puppet


## Process Flow

Query → Search FAISS → Get Relevant Chunk(s) → Pass to DistilBERT → Get Answer